# Climate

![](temperature.png)

Laten we een steentje bijdragen aan de klimaatdiscussie en data analyseren die door de ECA (European Climate Assessment) [beschikbaar](https://www.ecad.eu/dailydata/predefinedseries.php) wordt gemaakt in grote data files. We beperken ons tot data die de maximumtemperatuur beschrijft voor elke dag in De Bilt sinds 1901.

Werp eerst eens een blik op `climate.txt` (kies hierboven File->Open) en lees bovenin hoe de data gecodeerd is. Het bestand is vrij groot, dus het kan zijn dat Jupyter Notebooks er wat moeite mee heeft. Het bestand heeft geen standaard-formaat.

We willen onze data in het CSV-formaat krijgen. Dat is een fijn formaat dat je makkelijk kunt importeren in Excel, maar je kunt het ook goed inlezen met Python. De naam *comma-separated values* zegt het al: alle gegevens in dit formaat zijn gescheiden door een komma.

We gaan hieronder drie dingen doen:

1. het originele databestand opschonen en converteren naar een "echte" CSV (redelijk wat werk)
2. alle data inlezen vanuit de CSV (niet al te veel werk)
3. op basis van de data een aantal vragen beantwoorden (**veel werk**)

## Probleemanalyse

Voordat je gaat implementeren moet je voor alle onderdelen van de opdracht een probleemanalyse doen. Je hebt je databestand van ons gekregen, dus je weet hoe het eruit ziet. Allereerst moet je zorgvuldig analyseren wat het verschil is tussen het databestand en het gewenste CSV-bestand. Je kunt daarna ook eerst aan de slag met de analyse van de vragen die je inhoudelijk moet beantwoorden met hulp van de data. Zonder programmeren moet je al een algoritme kunnen formuleren voor hoe je elk van deze vragen kunt beantwoorden.

## Import

Om te beginnen vind je hier de import waarmee we straks de CSV-functionaliteit kunnen gebruiken. Zorg dat je de cel "runt" om te starten: selecteer de cel en klik hierboven op de knop **▶ Run**.

In [ ]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Preprocessing

Zoals gezegd zit het data bestand op een eigenwijze, niet-standaard manier in elkaar. We zullen deze eerst moeten opschonen (preprocessing), en dan de data weer in het gewenste formaat wegschrijven. Uiteindelijk zullen we `climate.txt` omzetten naar een CSV-bestand, waarin afzonderlijke waarden van elkaar gescheiden worden door komma's.

Het gewenste formaat van een CSV-bestand ziet er als volgt uit. De eerste regel moet de **namen** van alle kolommen bevatten, gescheiden door een komma. De volgende regels moeten alle datapunten bevatten, elk op één regel, waarbij alle waardes weer gescheiden worden door een komma. Hier zie je een CSV-file met twee kolommen:

![](telefoon.png)

Maar voordat we onze data op kunnen slaan in dit formaat, kijken we eerst zorgvuldig naar de data. Bestudeer hiervoor het bestand `climate.txt`. Het bestand `climate.txt` staat in dezelfde folder als dit notebook.

### 1.1 Header verwijderen (0.5 punt)

Wat meteen opvalt is dat het bestand begint met een header-tekstje. Dit tekstje komt van pas voor mensen die willen begrijpen hoe het bestand in elkaar steekt. Het bevat informatie over waar de data vandaan komt, maar ook over de betekenis van de variabelen. Voor een computer is deze tekst verwarrend, in die zin dat begin en eind niet duidelijk zijn. We besluiten daarom de header-tekst weg te halen, zodat het bestand voortaan begint met de kolomnamen.

Jouw opdracht is om een functie te schrijven die een aantal regels kan verwijderen van de top van een bestand.

![](step1.png)

- Schrijf in onderstaande cell een functie die een bestand opent, inleest, en wegschrijft onder een andere naam.

- De eerste regels moeten hierbij worden verwijderd, en de parameter `number_of_lines` geeft aan hoeveel regels moeten worden verwijderd.

- **Tip:** Start met de voorbeelden van de uitleg over File IO (hiermee heb je geoefend op de site).

- **Tip:** Om een regel te "verwijderen" moet je deze wél inlezen maar niet wegschrijven.

In [ ]:
def remove_header(number_of_lines, input_filename, output_filename):
    #TODO

remove_header(13, "climate.txt", "climate2.txt")

**🚨 Controleer voordat je verder gaat of de header-informatie is verwijderd in de file `climate2.txt` na het runnen van de functie hierboven. De kolomnamen `DATE` en `TX` moeten wel nog aanwezig zijn.**

### 1.2 Data filteren (0.5 punt)

Wie goed naar de bestanden kijkt ziet dat de data van het jaar 2020 niet compleet is. Als we dat jaar meenemen in de statistieken krijgen we mogelijk een vertekend beeld. Schoon het bestand dus verder op door alle gegevens van 2020 uit het bestand te halen. De functie die jij schrijft moet werken voor elk willekeurig jaartal, dus geef het jaartal mee als parameter van de functie.

![](step2.png)

In [ ]:
def removeYear(year, input_filename, output_filename):
    #TODO

removeYear(2020, "climate2.txt", "climate3.txt")

**Hint**: je bent nu specifiek geïnteresseerd in de datum, dus je zult naar de inhoud van de regel moeten kijken. Zo'n regel is een string, en je kent wel een techniek om de eerste vier tekens van een string op te vragen.

**🚨 Controleer voordat je verder gaat of de jaargegevens van 2020 zijn verwijderd in de file `climate3.txt` na het runnen van de functie hierboven. De laatste datum is dus `20191231`.**

### 1.3 Opschonen missende waarden

Tot slot bevat het bestand een aantal "missing values": datums waarvoor geen gegevens bekend zijn. In het oorspronkelijke databestand krijgen deze datums een TX-waarde van 9999. Deze gevallen kunnen de boel behoorlijk verstoren omdat ze een vertekend beeld kunnen geven als je statistieken wil berekenen.

--> We lossen dat in dit geval op met een functie `fill()` die deze ontbrekende waarden vervangt door de gemiddelde waarde van de volgende dag en de vorige dag.

![](step3.png)

De indicator voor een missing value verschilt per databestand; soms is dat een `-1`, soms een `0`, afhankelijk van de waarden die 'natuurlijk' in de data voorkomen. In het geval van deze data wordt `9999` gebruikt als indicator voor een missing value. Een veilige keuze, omdat de kans dat er een temperatuur van 999,9 graden gemeten wordt niet zo hoog is. Voor de functie `fill()` die je nu gaat schrijven is `missing_value` een parameter die aangeeft welke waarde het moet zijn.

De functie `fill()` is al voor je geschreven.

In [ ]:
def fill(missing_value, input_filename, output_filename):
    temperatures = []
    dates = []

    with open('climate3.txt', "r") as sourceFile, open("climate.csv", "w") as targetFile:
        targetFile.write(sourceFile.readline().replace(';', ',')) #Copy header
        for line in sourceFile:
            lineElements = line.split(';')
            dates.append(lineElements[0].strip())
            temperatures.append(int(lineElements[1].strip()))

        for index in range(len(temperatures)):
            if index == 0 and temperatures[index] == missing_value:
                temperatures[index] = temperatures[index+1]
            if index == len(temperatures)-1 and temperatures[index] == missing_value:
                temperatures[index] = temperatures[index-1]
            if temperatures[index] == missing_value:
                temperatures[index] = int((temperatures[index-1] + temperatures[index+1])/2)

        for index in range(len(temperatures)):
            targetFile.write(str(dates[index]) + ',' + str(temperatures[index]) + '\n')

fill(9999, "climate3.txt", "climate.csv")

**🚨 Controleer voordat je verder gaat of de indicator voor een missing value `9999` overal is verwijderd in de nieuwe file `climate.csv` na het runnen van de functie hierboven.**

## DictReader

Omdat je nu een CSV-bestand hebt dat netjes in het juiste formaat staat kun je heel handig een `csv.DictReader` gebruiken. Deze zorgt ervoor dat je voor een record alle informatie op **naam** kunt opzoeken. In onze dataset kun je dan bijvoorbeeld opvragen wat de DATE is, of wat de TX is. In het voorbeeld hieronder printen we alle kolomnamen (*field names*) en van elke regel de TX-waarde. Je kunt onderstaande code runnen om te checken of je CSV-bestand werkt.

In [ ]:
with open('climate.csv', "r") as sourceFile:
    reader = csv.DictReader(sourceFile)
    print(reader.fieldnames)
    for line in reader:
        print(line["TX"])

**Let op.** Het `climate.csv`-bestand moet gevuld zijn met data in CSV-formaat. Je moet ook de cel bovenaan (met de `import`s) een keer uitgevoerd hebben met **▶ Run** voordat je deze cel kunt runnen! Zeker als je gisteren bent gestopt en nu weer doorgaat met de opdracht.

**Mocht het nou niet gelukt zijn met het bewerken en wegschrijven van de data, dan is het mogelijk om alle bovenstaande stappen met de hand uit te voeren. In deze studie leer je data automatisch te verwerken en als bovenstaande je goed is afgegaan is dat een goed begin. Maar als het nog niet helemaal goed gegaan is, dan is het toegestaan het juiste CSV-bestand van iemand anders gebruiken zodat je verder kunt met onderstaande opdrachten.**

## 2. Hoe ziet de data er sinds het begin van de metingen uit? (1 punt)

Tijd voor wat analyse! We gaan onderzoeken wat de minimale, gemiddelde en maximale temperatuur gemeten is in De Bilt sinds het begin van de metingen, om een beeld te krijgen van de data. Schrijf in de cel hieronder code die `climate.csv` opent en doorleest en daarbij de minimale, gemiddelde en maximale temperatuur bijhoudt en uitprint. 

**Lopend minimum** Om zulke waarden te berekenen gebruik je de techniek van het lopende minimum. Terwijl je door de gegevens loopt kun je steeds weer het minimum *tot op dat moment* uitrekenen. Laten we deze gegevens nemen als voorbeeld:

|Naam|Lengte|
|:-|-|
|Arza|1.73|
|Bran|1.56|
|Croy|1.98|
|Damm|1.87|

Je kunt in Python het makkelijkste lopen door de data van boven naar beneden. Daar moeten we gebruik van maken. Je begint bij de eerste regel en dan kun je het minimum vastleggen dat alleen geldt voor die ene regel:

|Naam|Lengte|Lopend|
|:---|------|------|
|Arza|  1.73|  **1.73**|
|Bran|  1.56||
|Croy|  1.98||
|Damm|  1.87||

Vervolgens kun je het minimum vastleggen dat geldt voor regels 1 en 2. Je kijkt daarbij naar het minimum dat je al kende van regel 1, en je kijkt of de lengte op regel 2 *kleiner* is. In dit geval is dat zo, dus er is een nieuw minimum:

|Naam|Lengte|Lopend|
|:---|------|------|
|Arza|  1.73|  1.73|
|Bran|  1.56|  **1.56**|
|Croy|  1.98||
|Damm|  1.87||

De volgende regel bevat geen lengte die *kleiner* is, dus het oude minimum blijft behouden. Hetzelfde geldt voor de laatste regel:

|Naam|Lengte|Lopend|
|:---|------|------|
|Arza|  1.73|  1.73|
|Bran|  1.56|  1.56|
|Croy|  1.98|  **1.56**|
|Damm|  1.87|  **1.56**|

Na het aflopen van alle regels is de conclusie dat de minimumlengte **1.56** is. De truc is dat je hiervoor niet écht de complete tabel hoeft in te vullen. Je hebt namelijk voor elke berekening alleen maar de waarde van de *vorige regel* nodig en de waarde van de *huidige regel*.

🤔 Probeer nu de structuur voor een algoritme te bedenken dat het minimum kan berekenen. Je schrijft al een beetje pseudocode, een stappenplan, dat concreter is dan de beschrijving hierboven. Denk om te beginnen na wat de formule is voor het lopende minimum op een willekeurige regel en schrijf deze uit. Bespreek je idee met een medestudent, maar zorg dat je je eigen code schrijft. Dit is geen samenwerkopdracht.

✍️ Schrijf nu code in de cell hieronder. Op dezelfde manier kun je het maximum berekenen (en dit kan tegelijk!). Ook kun je het gemiddelde berekenen door alle regels af te lopen, dus alles kan gecombineerd worden in één loop.

In [ ]:
minimum = math.inf
gemiddelde = 0
maximum = -math.inf

# TODO: jouw uitwerking van het algoritme

print("Minimum: ", minimum)
print("Gemiddelde: ", gemiddelde)
print("Maximum: ", maximum)

**Startwaarden** Aan het begin van je code staan variabelen die het minimum, gemiddelde en maximum voorstellen. Deze moeten een goede beginwaarde krijgen. Het gemiddelde kan worden geïnitialiseerd op 0, maar voor het minimum en maximum hebben we waarden nodig die respectievelijk groter en kleiner zijn dan de laagste en hoogste voorkomende waarden in die data. Maar als we die waarden zouden hebben zouden we ze niet uit hoeven te rekenen! We kunnen wel een aanname doen over welke waarden in ieder geval passend zijn: `math.inf` is sowieso groter dan elk getal en `-math.inf` is kleiner dan elk willkeurig getal.

**Testen** Let op! In de cel hieronder staan "assertions" die testen of de variabelen wel voldoen aan een aantal basis-voorwaarden. Verander daarom de variabelenamen hierboven niet! De assertions werken dan niet goed meer. (Code die door de assertions komt hoeft overigens niet juist te zijn; de assertions testen alleen op de meest voor de hand liggende fouten.)

> Start de tests door de cel te runnen. Als de tests slagen, dan komt er **geen** output!

In [ ]:
assert gemiddelde >= minimum
assert gemiddelde <= maximum
assert maximum >= minimum
assert isinstance(minimum, int)
assert isinstance(maximum, int)

## 3. Mediaan en modus (1 punt)

Zoals je ziet, is het gemiddelde in het bovenstaande voorbeeld een redelijke inschatting van het middenpunt tussen minimum en maximum: het gemiddelde is hier een goede 'centrummaat'. Dat wordt deels veroorzaakt doordat de data *normaal verdeeld* is (meer daarover bij het vak Analysemethoden en -Technieken in de volgende periode). In sommige gevallen is het gemiddelde echter vrij gevoelig voor zogenaamde *outliers*, dus waarden die ongebruikelijk zijn. 

Stel bijvoorbeeld dat we bij de preprocessing waren vergeten om de missing values weg te halen; dan hadden ze met hun waarde van 9999 behoorlijk veel invloed op het gemiddelde gehad. Kortom: het gemiddelde is niet altijd een goede centrummaat. Er bestaan daarom twee andere veelgebruikte maten: de mediaan en de modus.

De **modus** is het meest voorkomende getal en de **mediaan** is het middelste getal als je alle getallen op volgorde van grootte zet. Bijvoorbeeld: van de reeks `9 3 1 7 1` is de modus 1 en de mediaan 3 (want het midden van `1 1 3 7 9`). Is dit voorbeeld je niet duidelijk, kijk dan op <https://nl.wikipedia.org/wiki/Modus_(statistiek)> of <https://nl.wikipedia.org/wiki/Mediaan_(statistiek)>.

Schrijf in de cel hieronder code die `climate.csv` opent, de modus en de mediaan berekent en uitprint. Het is niet toegestaan gebruik te maken van de module `statistics` of andere ingebouwde Python-functies om mediaan of modus te berekenen.

In [ ]:
mediaan = 0
modus = 0

# TODO: jouw uitwerking van de algoritmen

print("Mediaan: ", mediaan)
print("Modus: ", modus)

In [ ]:
assert mediaan >= minimum
assert mediaan <= maximum

assert modus >= minimum
assert modus <= maximum

assert isinstance(mediaan, int)
assert isinstance(modus, int)

**Interpretatievraag** Welke centrummaat geeft het beste het middenpunt tussen minimum en maximum weer? Vul je antwoord hieronder in.

> dubbelklik deze tekst en vul je antwoord in

We weten nu hoe de data er ongeveer uitziet. Nu gaan we er dieper induiken. Zo stellen we de vraag, warmt de aarde op? Om daarover uitspraken te doen over is het nodig de data per jaar te bekijken. Dat gaan we in de volgende opgave doen.

## 4. Wat zijn de maxima en minima per jaar? (1 punt)

In deze opgave gaan we de minimum- en maximum-waarden voor de temperatuur per jaar onderzoeken. We verwachten dat beide in de loop der tijd zijn toegenomen, maar zonder dat ook daadwerkelijk onderzocht te hebben kunnen we dat niet zomaar beweren.

> Eigenlijk kunnen we dat dan nog steeds niet beweren, want we onderzoeken hier alleen de meetwaarden van één station, maar het analyseren van de volledige data van het ECA zou hier te ingewikkeld worden.

Schrijf in de cel hieronder code die `climate.csv` opent en voor **alle jaren** sinds het begin van de metingen de volgende zin produceert:

    In 1918 varieerde de temperatuur tussen -0.8 graden op 04/02 en 30.3 graden op 22/08

Een manier om dit aan te pakken is door steeds alleen de gegevens per jaar in te lezen.

**Tips:** 

- Je mag er voor deze opgave vanuit gaan dat de temperaturen in de dataset op volgorde van datum worden gegeven. 
- Hou in een variabele bij met welk jaar je bezig bent terwijl je door de hele dataset heenloopt.

Let op: omdat je hier heel wat berekeningen moet doen, kan het (afhankelijk van je laptop) even duren voordat je resultaat ziet.

In [ ]:
# TODO: jouw uitwerking van deze opdracht

In onderstaande cel vind je code die de gegevens die je in deze vraag hebt gegenereerd omzet in een **scatterplot** (oftewel: een puntgrafiek). De code gaat ervanuit dat je de minimale waarden per jaar hebt opgeslagen in een lijst met de naam `minima` en de maximale waarden in een lijst met de naam `maxima`. Elk blauw stipje is een meetpunt en omdat het met het blote oog slecht zichtbaar is hebben we de trendontwikkeling van de temperatuur door de jaren heen in de figuur weergegeven. Kun je iets van een trend zien?

Let op! Ook dit deel van de opgave kan weer even duren.

In [ ]:
x = np.arange(1901, 2020, 1)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
ax1.set_xlabel('Year', fontdict={'fontsize' : 36})
ax1.set_ylabel('Minimum temperature (degrees celsius)', fontdict={'fontsize' : 36})
ax2.set_xlabel('Year', fontdict={'fontsize' : 36})
ax2.set_ylabel('Maximum temperature (degrees celsius)', fontdict={'fontsize' : 36})
fig.set_size_inches(35, 15)
sns.regplot(x=x, y=np.array(minima)/10, ax = ax1)
sns.regplot(x=x, y=np.array(maxima)/10, ax = ax2)

## 5. Hoe lang was per jaar de langste hittegolf? (1 punt)

Als we de berichten in de media mogen geloven dan gaat de klimaatverandering niet alleen gepaard met toenemende temperaturen, maar ook met extreme temperaturen. Of dat klopt onderzoeken we in deze vraag. Net als in de vorige vraag bepalen we **per jaar** hoe lang de langste hittegolf duurde.

> De definitie van een hittegolf is als volgt: voor een hittegolf moet het vijf aaneengesloten dagen tenminste 25 graden zijn, waarvan drie dagen tenminste 30 graden

Schrijf in de cel hieronder code die per jaar de lengte van de langste hittegolf geeft. Elke regel tekst volgt het volgende voorbeeld:

    In 1911 duurde de langste hittegolf 7 dagen

Wanneer er in een jaar geen hittegolf heeft plaatsgevonden, druk dat dan ook niet af.

In [ ]:
hittegolven = []

# TODO: jouw uitwerking van deze opdracht

In onderstaande cel vind je wederom code die de gegevens die je in deze vraag hebt gegenereerd omzet in een scatterplot. De code gaat ervanuit dat je de lengte van hittegolven hebt opgeslagen in een lijst met de naam `hittegolven`. Elk blauw stipje is een meetpunt en omdat het met het blote oog slecht zichtbaar is hebben we de trendontwikkeling van de lengte van hittegolven door de jaren heen in de figuur weergegeven.

Let op! Ook dit deel van de opgave kan weer even duren.

In [ ]:
x = np.arange(1901, 2020, 1)

ax = plt.gca()
ax.set_xlabel('Year', fontdict={'fontsize' : 24})
ax.set_ylabel('Lengte Hittegolven', fontdict={'fontsize' : 24})
sns.regplot(x=x, y=np.array(hittegolven), ax=ax)

## 6. Outliers detecteren (1 punt)

Je kent het waarschijnlijk wel: als er een koud jaar is, dan hoor je klimaatsceptici al snel roepen "Zie je wel: het valt wel mee met die klimaatverandering." Andersom is het natuurlijk ook waar: mensen die geloven dat de mens verantwoordelijk is voor de klimaatverandering zijn al snel geneigd een warme periode op te vatten als een onderbouwing daarvan, terwijl het al dan niet bestaan van klimaatverandering alleen op de lange termijn kan worden vastgesteld.

Elke dataset heeft outliers: datapunten met waarden die zo afwijken van wat we verwachten dat we ze liever niet meenemen in onze analyse. De officiele definitie is 'een statistische observatie die aantoonbaar afwijkt van de andere waarden'. Wat 'aantoonbaar' is, is echter voor discussie vatbaar. In deze opgave definiëren we outliers als punten die meer dan 3 'standaarddeviaties' afwijken van het gemiddelde. Voor diegenen die dit begrip niet kennen: de standaarddeviatie $\sigma$ is een maat voor de mate waarin data 'varieert', en wordt berekend met de volgende formule:

$$\sigma = \sqrt{\frac{\sum{(X-\mu)^2}}{N}}$$

In woorden: de standaarddeviatie is de wortel van de som ($\sum$) van het kwadraat van de verschillen van elk datapunt met het gemiddeld ($\mu$), gedeeld door het aantal datapunten *N*. N is het aantal meetwaarden in een jaar (meestal 365, maar vergeet de schrikkeljaren niet!).

Schrijf in de cel hieronder code die, gegeven data in de vorm van een lijst, in tekstvorm rapporteert wat de outliers zijn (als die er zijn). De tekst heeft het volgende formaat:

    Met 31.4 is de waarde op 03/06/1902 een outlier

Let op: deze opgave is wat ingewikkelder en het is daarom extra belangrijk dat je goed nadenkt over het 'ontwerp' van je opgave. Denk goed na over uit welke componenten je code moet bestaan, implementeer elke component op zich en maak je dan pas druk over de integratie van al je code. 

**Alvast een hint:** Omdat in deze opgave meerdere keren de data wordt doorlopen is het de moeite waard om die één keer in te lezen en dan weg te schrijven naar een lokale variabele.

In [ ]:
# TODO: jouw uitwerking van deze opdracht


## Tot slot

Notebooks hebben de vervelende eigenschap dat als je code verandert, dat niet meteen ververst wordt in de output van de rest van het notebook. Het kan zijn dat je denkt dat code goed is, maar dat je door een wijziging eerder in de file zonder dat je het doorhebt een bug hebt geïntroduceerd. Er is maar één manier om dat te voorkomen: maak er een gewoonte van om voordat je een notebook inlevert altijd eerst de Kernel (en daarmee het geheugen van de notebook) te herstarten en alle code opnieuw te draaien. Dat doe je door in het menu te kiezen voor **Kernel -> Restart & Run all**. Check dan van boven naar beneden de output. Is alles dan zoals je verwacht? Lever het dan gerust in in de wetenschap dat degene die het nakijkt hetzelfde ziet als jij.